In [11]:
#import all the necessary libraries
import pandas as pd
import os
import gzip
import re
import string
import json
from nltk.corpus import stopwords, reuters
from nltk import word_tokenize, FreqDist
from gensim.models import Word2Vec
pd.options.display.max_rows = 999

stop = stopwords.words('english')

os.getcwd()

'c:\\Users\\alann\\Documents\\DataScience\\Word2Vec_products'

In [13]:
#import the data
data = []
with gzip.open("meta_Electronics.json.gz") as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [14]:
#convert into data frame
df = pd.DataFrame.from_dict(data)
df.head(5)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Electronics, Camera &amp; Photo, Video Survei...",,[The following camera brands and models have b...,,Genuine Geovision 1 Channel 3rd Party NVR IP S...,[],[https://images-na.ssl-images-amazon.com/image...,,GeoVision,"[Genuine Geovision 1 Channel NVR IP Software, ...","[>#3,092 in Tools &amp; Home Improvement &gt; ...",[],Camera &amp; Photo,,"January 28, 2014",$65.00,0011300000,NaN
1,"[Electronics, Camera &amp; Photo]",,[This second edition of the Handbook of Astron...,,"Books ""Handbook of Astronomical Image Processi...",[0999470906],[https://images-na.ssl-images-amazon.com/image...,,33 Books Co.,[Detailed chapters cover these fundamental top...,"[>#55,933 in Camera &amp; Photo (See Top 100 i...","[0943396670, 1138055360, 0999470906]",Camera &amp; Photo,,"June 17, 2003",,0043396828,NaN
2,"[Electronics, eBook Readers &amp; Accessories,...",,[A zesty tale. (Publishers Weekly)<br /><br />...,,One Hot Summer,"[0425167798, 039914157X]",[],,Visit Amazon's Carolina Garcia Aguilera Page,[],"3,105,177 in Books (",[],Books,,,$11.49,0060009810,NaN
3,"[Electronics, eBook Readers & Accessories, eBo...",,[],,Hurray for Hattie Rabbit: Story and pictures (...,"[0060219521, 0060219580, 0060219394]",[],,Visit Amazon's Dick Gackenbach Page,[],"2,024,298 in Books (","[0060219521, 0060219475, 0060219394]",Books,,,.a-section.a-spacing-mini{margin-bottom:6px!im...,0060219602,NaN
4,"[Electronics, eBook Readers & Accessories, eBo...",,[&#8220;sex.lies.murder.fame. is brillllli&#82...,,sex.lies.murder.fame.: A Novel,[],[],,Visit Amazon's Lolita Files Page,[],"3,778,828 in Books (",[],Books,,,$13.95,0060786817,NaN


In [15]:
def pre_process(text):#Apply the function on each document
    tokens = nltk.word_tokenize(text) #convert the document into words
    tokens = [re.sub("[^a-zA-Z0-9]+","",t) for t in tokens]
    #tokens = [WNlemma.lemmatize(t) for t in tokens] #lemmatization the data
    tokens = [word for word in tokens if word not in string.punctuation ]
    tokens = [word.lower() for word in tokens]
    tokens = [word.strip() for word in tokens]
    #tokens = [re.sub("^\d{4}(hrs)$|^\d{4}(hr)$|\d+$","",t) for t in tokens] #remove time
    tokens = [word for word in tokens if word not in stop and len(word) > 3] #remove the stop words and those length more than 3
    text_after_process = " ".join(tokens) #join the tokens together to form back the document
    return(text_after_process)

#This portion can be further improved with bigram to match longer words.

In [16]:
df['trans_title'] = df['title'].apply(pre_process)

In [17]:
#check which word count is the highest
unlist_text = nltk.word_tokenize(df.trans_title.str.cat(sep = ' '))

In [18]:
fd = nltk.FreqDist(unlist_text)
fd.most_common(50)

[('case', 131584),
 ('quot', 103037),
 ('black', 100397),
 ('cable', 78754),
 ('camera', 74694),
 ('ipad', 70145),
 ('cover', 61099),
 ('adapter', 60904),
 ('laptop', 57481),
 ('battery', 56741),
 ('digital', 53355),
 ('mini', 51326),
 ('power', 46530),
 ('screen', 44147),
 ('replacement', 42632),
 ('samsung', 42383),
 ('wireless', 41940),
 ('charger', 41639),
 ('tablet', 41611),
 ('bluetooth', 39008),
 ('inch', 38282),
 ('apple', 37758),
 ('card', 36682),
 ('sony', 35192),
 ('lens', 35080),
 ('leather', 32438),
 ('drive', 31106),
 ('white', 30057),
 ('series', 28955),
 ('galaxy', 28528),
 ('stand', 28474),
 ('hard', 28228),
 ('canon', 28054),
 ('audio', 27806),
 ('memory', 27789),
 ('keyboard', 26596),
 ('speaker', 26191),
 ('compatible', 26101),
 ('touch', 25881),
 ('protector', 25679),
 ('video', 25609),
 ('cord', 25285),
 ('pack', 25265),
 ('macbook', 25145),
 ('stereo', 25023),
 ('blue', 24334),
 ('ipod', 24292),
 ('mount', 24104),
 ('high', 23905),
 ('iphone', 22470)]

In [65]:
removed_words = ['psia']
df['trans_title']

0         genuine geovision channel party software dongl...
1         books handbook astronomical image processing e...
2                                                    summer
3         hurray hattie rabbit story pictures early read...
4                                   sexliesmurderfame novel
                                ...                        
786440    tukzer cassette player portable tape player ca...
786441    cobiter adapter battery charger lenovo thinkpa...
786442    baofeng original liion battery charger desktop...
786443    jricoo 3pack micro cable nylon braided extreme...
786444    jricoo nylon braided micro charging cable feet...
Name: trans_title, Length: 786445, dtype: object

In [63]:
train = df['trans_title'].str.split(' ')

In [102]:
model = Word2Vec(train, min_count=10, window = 3, workers = 4, size = 50)

In [103]:
print(model)  

Word2Vec(vocab=38561, size=50, alpha=0.025)


In [104]:
words = list(model.wv.vocab)

In [105]:
print(words)

['genuine', 'geovision', 'channel', 'party', 'software', 'dongle', 'onvif', 'books', 'astronomical', 'image', 'processing', 'edition', 'hardcover', 'book', 'berry', 'summer', 'rabbit', 'story', 'pictures', 'early', 'read', 'novel', 'college', 'physics', 'girl', 'mind', 'next', 'door', 'abcgoodefg', 'music', 'player', 'radio', 'function', 'voice', 'wireless', 'bluetooth', 'headphones', 'earbuds', 'microphone', 'noise', 'cancelling', 'sweatproof', 'earphones', 'stereo', 'headset', 'running', 'workout', 'sports', 'secure', 'case', 'training', 'mastering', 'blend', 'modes', 'adobe', 'photoshop', 'last', 'life', 'lady', 'practical', 'modern', 'basketball', 'samsung', 'plus', 'microsdxc', 'class', 'memory', 'card', 'boox', 'carta', 'flexible', 'screen', 'builtin', 'wifi', 'premium', 'performance', 'camcorder', 'videotape', '6pack', 'lord', 'rings', 'return', 'king', 'extended', 'pattern', 'black', 'angel', 'doll', 'christmas', 'money', 'never', 'first', 'year', 'review', '2008', 'crash', 'co

In [109]:
model.most_similar(positive=['surface'], topn=5)

<ipython-input-109-1a58edf025e6>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['surface'], topn=5)


[('microsoft', 0.6374869346618652),
 ('kinect', 0.5689845085144043),
 ('4pro', 0.5670135021209717),
 ('prosurface', 0.5660805702209473),
 ('rtsurface', 0.5509934425354004)]

In [111]:
model.save('model.bin')